In [7]:
import pandas as pd
import numpy as np
from app import *



  

In [8]:
   
tickers = ["AAPL", "MSFT", "GOOGL", "AMZN", "META", "TSLA", "NVDA"]



def acc(arg):
    try:
        symbol = arg
        stock = yf.Ticker(symbol)
        if market_open():
                stock_price = stock.history(period="1d")["Close"].iloc[-1]
        else:
                stock_price = stock.history(period="5d")["Close"].iloc[-1] 
        name = stock.info.get("longName", "Company name not found")
        expiring_dates = stock.options
        early_date = expiring_dates[0]  # Getting the first expiring date
        options_chain = stock.option_chain(early_date)
        calls = options_chain.calls
        puts = options_chain.puts
        strike_price_calls = calls['strike'].iloc[0]
        strike_price_puts = puts['strike'].iloc[0]
        today = datetime.today().date()
        early_date_obj = datetime.strptime(early_date, '%Y-%m-%d').date()
        time_to_expiry = (early_date_obj - today).days / 365
        if time_to_expiry <= 0:
            print("Expiration date is today or has already passed. Adjusting T to 1 day.")
            time_to_expiry = 1 / 365
        implied_volatility_calls = calls[calls['strike'] == strike_price_calls]['impliedVolatility'].values[0]
        implied_volatility_puts = puts[puts['strike'] == strike_price_puts]['impliedVolatility'].values[0]
        treasury_data = yf.Ticker("^IRX")
        latest_data = treasury_data.history(period="1d")
        treasury_yield = latest_data['Close'].iloc[-1]
        risk_free_rate = treasury_yield / 100
        option_premium_call = calls[calls['strike'] == strike_price_calls]['lastPrice']
        option_premium_puts = puts[puts['strike'] == strike_price_puts]['lastPrice']
        optimized_implied_volatility = sentiment_analysis(symbol)
        print("hi", sentiment_analysis(symbol))
        print("Today:", today)
        print("Early Date Object:", early_date_obj)
        print("Difference (days):", (early_date_obj - today).days)

        new_volitlity_calls = implied_volatility_calls * (1 + 2 * (optimized_implied_volatility - 0.5))
        S = stock_price
        K = strike_price_calls
        r = risk_free_rate
        T = time_to_expiry
        if T == 0:  # Time will go to 0 on weekends so made this temp fix :)
            T = 1
        if K == 0:
            print("this is K", K)
            

        sigma = implied_volatility_calls
        option_type = "call"
        if sigma == 0:
            print("this is sigma", sigma)
            

        price = black_scholes_price(S, K, r, T, sigma, option_type)
        greeks = black_scholes_greeks(S, K, r, T, sigma, option_type)
        optimized_price = black_scholes_price(S, K, r, T, new_volitlity_calls, option_type)

        result = {
            "name": name,
            "stock price": stock_price,
            "time to maturity": time_to_expiry,
            "implied volatility": implied_volatility_calls,
            "risk free rate": risk_free_rate,
            "call strike price": strike_price_calls,
            "price": price,
            "Real-world premium": option_premium_call.values[0],
            "Optimized premium": optimized_price
        }

        vals = [option_premium_call.values[0], optimized_price, price]
        return vals
    except Exception as e:
        return e

list_vals = []          
# for ticker in tickers:
#     #val = acc(ticker)   
#     list_vals.append(val)   

print(acc("AAPL"))       

   
   
   
   
   

    

name 'market_open' is not defined


In [6]:
averages_i = []
averages = []
for i in list_vals:
    avg_optimized = 100 - (abs(i[0]-i[1])/i[0])*100
    avg_black = 100 - (abs(i[0]-i[2])/i[0])*100
    averages_i = [avg_optimized]
    averages_i.append(avg_black)
    averages.append(averages_i)
    averages_i = []
print(averages)



[]
